In [3]:
import xgboost as xgb

In [4]:
from sklearn.datasets import load_boston
boston = load_boston()

In [6]:
import pandas as pd

data = pd.DataFrame(boston.data)
data.columns = boston.feature_names
data['PRICE'] = boston.target
data.head()

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,PRICE
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,296.0,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2.0,242.0,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3.0,222.0,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3.0,222.0,18.7,396.90,5.33,36.2


In [8]:
x=data.drop(columns="PRICE")
y=data[["PRICE"]]

In [17]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=123)

In [9]:
from sklearn.model_selection import GridSearchCV

In [20]:
grdisearch=GridSearchCV(estimator=xgb.XGBRegressor(),
             param_grid={'learning_rate': [.1,.2,.5,1], 'max_depth': [2,4,3,6],'subsample':[.1,.2,.3,.4,.6,.7],
                        'colsample_bytree':[.1,.2,.3,.4],'n_estimators':[40,10,20,50,100],
                        'lambda':[10,20,30,50],'gamma':[10,50,20]})

In [21]:
grdisearch=grdisearch.fit(X_train,y_train)

In [22]:
grdisearch.best_params_

{'colsample_bytree': 0.4,
 'gamma': 10,
 'lambda': 10,
 'learning_rate': 0.2,
 'max_depth': 6,
 'n_estimators': 100,
 'subsample': 0.7}

In [220]:
model1=xgb.XGBRegressor(objective="reg:linear",colsample_bytree=0.4,reg_alpha=5,gamma=25,reg_lambda=10,learning_rate=.2,max_depth=6,n_estimators=100,subsample= 0.7)

In [221]:
model1.fit(X_train,y_train)

[14:42:12] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.1.0/src/objective/regression_obj.cu:170: reg:linear is now deprecated in favor of reg:squarederror.
[14:42:12] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.1.0/src/objective/regression_obj.cu:170: reg:linear is now deprecated in favor of reg:squarederror.


XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=0.4, gamma=25, gpu_id=-1,
             importance_type='gain', interaction_constraints='',
             learning_rate=0.2, max_delta_step=0, max_depth=6,
             min_child_weight=1, missing=nan, monotone_constraints='()',
             n_estimators=100, n_jobs=0, num_parallel_tree=1,
             objective='reg:linear', random_state=0, reg_alpha=5, reg_lambda=10,
             scale_pos_weight=1, subsample=0.7, tree_method='exact',
             validate_parameters=1, verbosity=None)

In [222]:
data_dmatrix = xgb.DMatrix(data=y,label=y)
data_dmatrix

In [223]:
trainpredict=model1.predict(X_train)

In [224]:
testpredict=model1.predict(X_test)

In [225]:
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error

In [226]:
print(r2_score(y_train,trainpredict))
mean_squared_error(y_train,trainpredict)

0.9464478988018069


4.5369741080284065

In [227]:
print(r2_score(y_test,testpredict))
mean_squared_error(y_test,testpredict)

0.8113998101561446


15.603979190150469

In [228]:
params = {"objective":"reg:linear",'colsample_bytree': 0.4,'learning_rate': 0.2,
                'max_depth': 6, 'reg_alpha': 15,'gamma':10,"reg_lambda":10,'n_estimators':100,'subsample': 0.7}

In [229]:
from sklearn.model_selection import cross_val_score

In [235]:
cv_results = xgb.cv(dtrain=data_dmatrix, params=params, nfold=3,
                    num_boost_round=50,early_stopping_rounds=10,metrics="rmse", as_pandas=True, seed=123)

[14:43:26] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.1.0/src/objective/regression_obj.cu:170: reg:linear is now deprecated in favor of reg:squarederror.
[14:43:26] WARNING: C:\Users\Administrator\workspace\xgboost-win64_release_1.1.0\src\learner.cc:480: 
Parameters: { n_estimators } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[14:43:26] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.1.0/src/objective/regression_obj.cu:170: reg:linear is now deprecated in favor of reg:squarederror.
[14:43:26] WARNING: C:\Users\Administrator\workspace\xgboost-win64_release_1.1.0\src\learner.cc:480: 
Parameters: { n_estimators } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to